In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName('SemiJoin').setMaster('yarn')\
   .set('spark.sql.legacy.timeParserPolicy', 'LEGACY')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/30 21:11:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/30 21:11:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/30 21:11:54 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/10/30 21:11:55 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                   application_1727681258360_0021


In [7]:
emp = [(1, 'Anna'), (2, 'Bill'), (3, 'John'), (4, 'Kate')]
 
df_emp = spark.createDataFrame(data=emp, schema=['emp_id', 'name'])
 
empHistPos = [ (1, 'Jr Developer', 'PROMOTED'),
               (1, 'Developer', 'PROMOTED'),
               (1, 'Sr Developer', ''),
                  (2, 'Jr Manager', ''),
               (3, 'Jr Sales Repr', 'PROMOTED'),
               (3, 'Sales Repr', 'PROMOTED'),
               (3, 'Sr Sales Repr', ''),
                  (4, 'Jr HR Manager', 'PROMOTED'),
                  (4, 'HR Manager', 'FIRED')]
 
df_hist_pos = spark.createDataFrame(data=empHistPos, schema=['emp_id', 'pos_name', 'change_reason'])
 
df_promoted = df_hist_pos.filter("change_reason = 'PROMOTED'") # double records
 
df_emp_promoted = df_emp.join(df_promoted, 'emp_id', 'leftsemi')
 
df_emp_promoted.orderBy('emp_id').show(truncate=False)

+------+----+
|emp_id|name|
+------+----+
|1     |Anna|
|3     |John|
|4     |Kate|
+------+----+



In [8]:
# df_emp_active = df_emp_hist_pos.filter("change_reason = ''")
# df_emp_active = df_emp.join(df_emp_fired, 'emp_id', 'leftanti').orderBy('emp_id')

# df_emp_promoted = df_emp.join(df_promoted, 'emp_id', 'left').orderBy('emp_id')
# df_res = df_emp.join(df_emp_hist_pos_active, 'emp_id', 'inner').select('emp_id', 'name', 'pos_name').orderBy('emp_id')


spark.stop()
# df_list = spark.sql(" select repeat(',', {num_days}) as seq".format(num_days=31))\
    # .withColumn('arr', split(col('seq'), ','))\